In [1]:
# All required python standard libraries
import os
import time

In [2]:
# All torch related imports 
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from torch import nn, optim

In [3]:
# using cv2 to read an image
import cv2

In [4]:
# All sci-kit related imports 
import pandas as pd
import numpy as np

In [5]:
from tqdm.notebook import tqdm as tq

In [6]:
train_directory = os.path.join(os.getcwd(),'guides\\isolated-dataset-csv\\IsolatedTrain.csv')
test_directory = os.path.join(os.getcwd(),'guides\\isolated-dataset-csv\\IsolatedTest.csv')

In [7]:
BATCH_SIZE = 64

In [8]:
train_csv = pd.read_csv(train_directory, usecols=["labels","directory"])
test_csv = pd.read_csv(test_directory, usecols=["labels","directory"])

In [9]:
def csv_preprocessor(base_dir:str, directory:str):
    return os.path.join(base_dir,directory).replace("\\","/")

In [10]:
def return_all_image_list_from_processed_csv(csv_file):
    ### This returns the entire list full of images to be loaded into cpu
    ###
    ###
    ALL_IMAGES = []
    start = time.time()
    for i, items in tq(enumerate(csv_file.iloc[:,1])):
        image = cv2.imread(items, cv2.COLOR_BGR2RGB)
        resized = cv2.resize(image,(224,224))
        ALL_IMAGES.append(resized)
    
    print("Tt took us approximately {} seconds".format(time.time()-start))  
    return ALL_IMAGES

In [11]:
class IsolatedCharacterDataset(Dataset):
    def __init__(self, csv_dir_path,  transforms=None, custom_transform=None ):
        ### complete dataset path
        self.dataset_csv = pd.read_csv(csv_dir_path, usecols=["labels","directory"])  
        self.dataset_csv["directory"] = self.dataset_csv["directory"].map(lambda x: csv_preprocessor(base_dir=str(os.getcwd()), directory=str(x)))
        self.dataset_csv_numpy = self.dataset_csv.to_numpy()
        self.ALL_IMAGES = return_all_image_list_from_processed_csv(csv_file=self.dataset_csv)
        
        ### labels
        self.labels = self.dataset_csv_numpy[:,0]
        ### images directories
        self.image_directories = self.dataset_csv_numpy[:,1]
        ### transformations to apply on images
        self.transforms = transforms
        
    def __getitem__(self, index):
        # convert labels to tensor 
        label = torch.tensor(self.labels[index])
        
        # load single image from list of all preloaded images
        image = self.ALL_IMAGES[index]
        
        if self.transforms:
            ## apply transforms 
            image = self.transforms(image)
            image = image.float()
        label = label.long().subtract(1)
        return image, label 
    
    def __len__(self):
        rows,_ = self.dataset_csv_numpy.shape
        return rows

In [12]:
DATA_NORMALIZER = transforms.Compose([transforms.ToTensor(),transforms.Resize(224),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),])

In [13]:
TRAIN_DATASET = IsolatedCharacterDataset(csv_dir_path= train_directory,transforms=DATA_NORMALIZER)
TRAIN_LOADER = DataLoader(dataset=TRAIN_DATASET,batch_size=BATCH_SIZE,num_workers=0,shuffle= True)


Tt took us approximately 547.6679956912994 seconds


In [14]:
for x in range(1):
    start = time.time()
    for i,data in enumerate(TRAIN_LOADER):
        load_start = time.time()
        image, label = data
        print("---> batch no  {}, time taken {}".format(i, time.time()-load_start ))
    print("pass {} time taken {}".format(x, -start + time.time()))

---> batch no  0, time taken 0.0
---> batch no  1, time taken 0.0019941329956054688
---> batch no  2, time taken 0.0019941329956054688
---> batch no  3, time taken 0.001994609832763672
---> batch no  4, time taken 0.0019941329956054688
---> batch no  5, time taken 0.0019943714141845703
---> batch no  6, time taken 0.0029897689819335938
---> batch no  7, time taken 0.001993894577026367
---> batch no  8, time taken 0.001994609832763672
---> batch no  9, time taken 0.0019943714141845703
---> batch no  10, time taken 0.0019943714141845703
---> batch no  11, time taken 0.0019941329956054688
---> batch no  12, time taken 0.0019948482513427734
---> batch no  13, time taken 0.001994609832763672
---> batch no  14, time taken 0.001994609832763672
---> batch no  15, time taken 0.001994609832763672
---> batch no  16, time taken 0.0019943714141845703
---> batch no  17, time taken 0.001993894577026367
---> batch no  18, time taken 0.0029916763305664062
---> batch no  19, time taken 0.001994848251342

---> batch no  158, time taken 0.002992391586303711
---> batch no  159, time taken 0.001993894577026367
---> batch no  160, time taken 0.007977724075317383
---> batch no  161, time taken 0.0019943714141845703
---> batch no  162, time taken 0.0019948482513427734
---> batch no  163, time taken 0.002993345260620117
---> batch no  164, time taken 0.0019943714141845703
---> batch no  165, time taken 0.0019941329956054688
---> batch no  166, time taken 0.0019941329956054688
---> batch no  167, time taken 0.0019941329956054688
---> batch no  168, time taken 0.002991199493408203
---> batch no  169, time taken 0.0029916763305664062
---> batch no  170, time taken 0.0019948482513427734
---> batch no  171, time taken 0.0019941329956054688
---> batch no  172, time taken 0.0019936561584472656
---> batch no  173, time taken 0.0029916763305664062
---> batch no  174, time taken 0.0019941329956054688
---> batch no  175, time taken 0.0019943714141845703
---> batch no  176, time taken 0.001994609832763672

---> batch no  317, time taken 0.002991199493408203
---> batch no  318, time taken 0.001994609832763672
---> batch no  319, time taken 0.0019943714141845703
---> batch no  320, time taken 0.0019943714141845703
---> batch no  321, time taken 0.0019948482513427734
---> batch no  322, time taken 0.0019943714141845703
---> batch no  323, time taken 0.002992868423461914
---> batch no  324, time taken 0.0019943714141845703
---> batch no  325, time taken 0.0019941329956054688
---> batch no  326, time taken 0.0029754638671875
---> batch no  327, time taken 0.0019948482513427734
---> batch no  328, time taken 0.0019969940185546875
---> batch no  329, time taken 0.001995086669921875
---> batch no  330, time taken 0.002991914749145508
---> batch no  331, time taken 0.002986431121826172
---> batch no  332, time taken 0.0019948482513427734
---> batch no  333, time taken 0.001994609832763672
---> batch no  334, time taken 0.001995086669921875
---> batch no  335, time taken 0.0019941329956054688
--->

---> batch no  474, time taken 0.002991914749145508
---> batch no  475, time taken 0.0019943714141845703
---> batch no  476, time taken 0.001993894577026367
---> batch no  477, time taken 0.0019941329956054688
---> batch no  478, time taken 0.0019943714141845703
---> batch no  479, time taken 0.001993894577026367
---> batch no  480, time taken 0.0019943714141845703
---> batch no  481, time taken 0.001994609832763672
---> batch no  482, time taken 0.001993417739868164
---> batch no  483, time taken 0.0019943714141845703
---> batch no  484, time taken 0.001994609832763672
---> batch no  485, time taken 0.0029914379119873047
---> batch no  486, time taken 0.001993894577026367
---> batch no  487, time taken 0.0029900074005126953
---> batch no  488, time taken 0.002991199493408203
---> batch no  489, time taken 0.002991914749145508
---> batch no  490, time taken 0.0019948482513427734
---> batch no  491, time taken 0.003988504409790039
---> batch no  492, time taken 0.002991914749145508
--->

In [ ]:
def training(epochs:int):
    all_training_losses = []
    all_training_accuracy = []
    for epoch in tqdm(range(epochs)):
        total_epoch_loss = 0
        total_accuracy_epoch = 0
        for i, data in tqdm(enumerate(TRAIN_LOADER, 0)): 
            image,label = data
            optimizer.zero_grad(set_to_none=True)
            
            label = label.to(DEVICE)
            image = image.to(DEVICE)
            output = net(image)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                total_epoch_loss += loss
                batches_training_accuracy = accuracy_finder(predictions=output, labels=label)
                total_accuracy_epoch = total_accuracy_epoch  + batches_training_accuracy   
            
            if i % 1000 == 0: 
               print("Batch : {}/{}".format(i, len(TRAIN_LOADER))) 
        # total epoch loss 
        total_epoch_loss = total_epoch_loss / len(TRAIN_LOADER)
        # total epoch accuracy 
        total_accuracy_epoch = total_accuracy_epoch /len(TRAIN_LOADER)
        
        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.8f}, acc = {:.8f}".format(epoch + 1, epochs, total_epoch_loss, total_accuracy_epoch ))
        all_training_losses.append(total_epoch_loss)
        all_training_accuracy.append(total_accuracy_epoch)
        
    print("Training completed")
    return all_training_accuracy, all_training_losses

In [ ]:
t_acc, t_loss = training(100)